In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
twitter=pd.read_csv('/kaggle/input/sentimental-analysis-for-tweets/sentiment_tweets3.csv')

In [ ]:
twitter.head(10)

In [ ]:
twitter.info()

In [ ]:
twitter=twitter.drop('Index',axis=1)

In [ ]:
twitter.sample(10)

In [ ]:
twitter['message to examine'][9680]

In [ ]:
# import re
# re.sub('([a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','', twitter['message to examine'][4], flags=re.MULTILINE)

In [ ]:
import re
for i in range(len(twitter['message to examine'])):
    twitter['message to examine'][i]=re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','', twitter['message to examine'][i], flags=re.MULTILINE)

In [ ]:
twitter

In [ ]:
for p in range(len(twitter['message to examine'])):
    twitter['message to examine'][p]=re.sub('@([a-zA-Z])+','', twitter['message to examine'][p], flags=re.MULTILINE)

In [ ]:
twitter

In [ ]:
sum=0
li=[]
for k in range(len(twitter)):
    if twitter['message to examine'][k].isdigit()==True:
        sum+=1
        li.append(k)
sum

In [ ]:
li

In [ ]:
su=0
for o in range(len(twitter)):
    if len(twitter['message to examine'][o])<6:
        su+=1
        li.append(o)
su

In [ ]:
li

In [ ]:
twitter=twitter.drop(index=li)

In [ ]:
twitter.reset_index(inplace=True)

In [ ]:
twitter

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(twitter['index'],twitter['label (depression result)'])

In [ ]:
zerolabel=twitter[twitter['label (depression result)']==0]

In [ ]:
onelabel=twitter[twitter['label (depression result)']==1]

In [ ]:
zerolabel=zerolabel.sample(frac=1)

In [ ]:
onelabel=onelabel.sample(frac=1)

In [ ]:
train=pd.concat([zerolabel[:(len(zerolabel)*3)//5],onelabel[:(len(onelabel)*3)//5]])

In [ ]:
train=train.sample(frac=1)

In [ ]:
train=train.drop(columns='index')

In [ ]:
train.reset_index(inplace=True)

In [ ]:
train=train.drop(columns='index')

In [ ]:
test=pd.concat([zerolabel[(len(zerolabel)*3)//5:],onelabel[(len(onelabel)*3)//5:]])

In [ ]:
test=test.sample(frac=1)

In [ ]:
test=test.drop(columns='index')
test.reset_index(inplace=True)
test=test.drop(columns='index')

In [ ]:
X_train=train.drop(columns=['label (depression result)'])
y_train=(train['label (depression result)'])
X_test=test.drop(columns=['label (depression result)'])
y_test=(test['label (depression result)'])

In [ ]:
X_train.reset_index(inplace=True)

In [ ]:
X_train

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(0, len(X_train)):
    review = re.sub('[^a-zA-Z]', ' ', X_train['message to examine'][i])
    review = review.lower()
    review = review.split()
    
    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
X_test.reset_index(inplace=True)

In [ ]:
corptest = []
for i in range(0, len(X_test)):
    review = re.sub('[^a-zA-Z]', ' ', X_test['message to examine'][i])
    review = review.lower()
    review = review.split()
    
    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corptest.append(review)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
cv = TfidfVectorizer(max_features=8000,ngram_range=(1,3))
X_train = cv.fit_transform(corpus).toarray()
X_test=cv.transform(corptest).toarray()

In [ ]:
y_test=np.array(y_test)
y_train=np.array(y_train)

In [ ]:
from sklearn.linear_model import PassiveAggressiveClassifier
linear_clf = PassiveAggressiveClassifier()

In [ ]:
linear_clf.fit(X_train,y_train)

In [ ]:
y_pred=linear_clf.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
accuracy_score(y_test,y_pred)

In [ ]:
import seaborn as sns
sns.heatmap(confusion_matrix(y_test,y_pred),annot=True,fmt='.4g')

In [ ]:
classification_report(y_test,y_pred)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
classifier=MultinomialNB()

In [ ]:
classifier.fit(X_train,y_train)
y_np=classifier.predict(X_test)

In [ ]:
accuracy_score(y_test,y_np)

In [ ]:
classification_report(y_test,y_np)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
randomclassifier=RandomForestClassifier(n_estimators=200,criterion='entropy')
randomclassifier.fit(X_train,y_train)

In [ ]:
y_predictor=randomclassifier.predict(X_test)

In [ ]:
accuracy_score(y_test,y_predictor)

In [ ]:
sns.heatmap(confusion_matrix(y_test,y_predictor),annot=True,fmt='.4g')

In [ ]:
classification_report(y_test,y_predictor)